
<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img
    src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png"
    alt="Databricks Learning"
  >
</div>


# 3.2 DEMO: Fine Grained Access Control using Dynamic Views and Partition Filtering \[Provider]

## Overview
This demonstration showcases two key data governance patterns for Delta Sharing:

- **Dynamic Views**: Row-level security based on recipient identity using `current_recipient()`
- **Partition Filtering**: Region-based data access control through table partitioning

**Provider Notebook (This Notebook):** Create source data, implement one dynamic view example and one partition filtering example, then share with recipients.

**Recipient Notebook:** Access shared views as different recipients and observe how access controls work.

### Learning Objectives
By the end of this demo, you will be able to:
1. Implement row-level security using dynamic views with `current_recipient()`
2. Create partition-based filtering for regional data boundaries
3. Share views with recipient-specific access controls

## Background

**Scenario:** You are "GlobalRetail Inc" with a sales database containing customer orders across multiple regions (North America, Europe, Asia Pacific). You need to share data with regional sales partners who should only see data from their respective regions.

In [0]:
%run ./Includes/Demo-Setup-3.2

## Example 1: Dynamic Views with Row-Level Security

Dynamic views use the `current_recipient()` function to filter data based on who is accessing it.  See the following diagram:  
<br />
<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img
    src="https://github.com/stackql/databricks-data-sharing-and-collaboration/blob/main/images/dynamic-views.png?raw=true"
    alt="Dynamic Views"
  >
</div>
<br />
<br />

ℹ️ **IMPORTANT** Dynamic Views will use compute on the provider side when queried by the Delta Sharing recipient.

### Inspect Base Table Data
First we will look at the source table to develop our row filtering or column redaction strategy.

In [0]:
-- Inspect the source table data
SELECT * FROM global_retail.sales.customer_orders ORDER BY region;

### Step 1. Create the Dynamic View
Dynamic views can be used to secure data in multiple ways including:  
- column redaction or obfuscation - using a `CASE` statement with the `current_recipient()` function in the `SELECT` column list; **and/or**
- row level filtering - using a `CASE` statement with the `current_recipient()` function in the `WHERE` clause

In this example we will filter records in the source table based upon the region, using recipients designated to specific regions.

In [0]:
-- Create dynamic view that filters by recipient
CREATE OR REPLACE VIEW global_retail.sales.regional_orders_view AS
SELECT 
  order_id,
  customer_name,
  email,
  region,
  product,
  amount,
  order_date
FROM global_retail.sales.customer_orders
WHERE 
  CASE 
    WHEN current_recipient() = 'us_partner' THEN region = 'United States'
    WHEN current_recipient() = 'eu_partner' THEN region = 'Europe'
    WHEN current_recipient() = 'apac_partner' THEN region = 'Asia Pacific'
    WHEN current_recipient() IS NULL THEN TRUE  -- Direct query (all data)
    ELSE FALSE  -- Unknown recipient (no access)
  END;

In [0]:
-- When queried directly (current_recipient() is NULL), we see all records
SELECT * FROM global_retail.sales.regional_orders_view;

**Key Concept:** The `current_recipient()` function returns the recipient name when data is accessed through Delta Sharing, enabling automatic row-level filtering. When queried directly, it returns `NULL`.

### Step 2. Create a Share
For Dynamic Views we can create a single share.

In [0]:
-- Create share for dynamic views
CREATE SHARE IF NOT EXISTS regional_orders_dynamic
COMMENT 'Orders with dynamic view filtering';

In [0]:
-- Describe the share
DESCRIBE SHARE regional_orders_dynamic;

### Step 3. Add the Dynamic View to the Share
Add our Dynamic View to the Share we just created.

In [0]:
-- Add the Dynamic View to the Share
ALTER SHARE regional_orders_dynamic 
ADD VIEW global_retail.sales.regional_orders_view;

In [0]:
-- Show assets in the share
SHOW ALL IN SHARE regional_orders_dynamic;

### Step 4. Create the Recipients
Create *multiple* recipients for our Share

In [0]:
-- Create recipients
CREATE RECIPIENT IF NOT EXISTS us_partner
COMMENT 'United States regional partner';

CREATE RECIPIENT IF NOT EXISTS eu_partner
COMMENT 'Europe regional partner';

CREATE RECIPIENT IF NOT EXISTS apac_partner
COMMENT 'Asia Pacific regional partner';

In [0]:
-- Describe one of the recipients
DESCRIBE RECIPIENT us_partner;

### Step 5. Add the Dynamic View to the Share
Add our Dynamic View to the Share we just created.

In [0]:
-- Grant access to all partners (filtering handled by view logic)
GRANT SELECT ON SHARE regional_orders_dynamic TO RECIPIENT us_partner;
GRANT SELECT ON SHARE regional_orders_dynamic TO RECIPIENT eu_partner;
GRANT SELECT ON SHARE regional_orders_dynamic TO RECIPIENT apac_partner;

In [0]:
-- Show grants on share
SHOW GRANTS ON SHARE regional_orders_dynamic;

### Next Steps
✅ Mount the share to a catalog at a recipient (one of the recipients above)  
✅ Confirm that you can only see data for your region  

## Example 2: Partition Filtering

Partition filtering leverages table partitioning to restrict data access at the storage level, providing efficient region-based access control.  See the following diagram:  
<br />
<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img
    src="https://github.com/stackql/databricks-data-sharing-and-collaboration/blob/main/images/partition-filtering.png?raw=true"
    alt="Partition Filtering"
  >
</div>
<br />
<br />

ℹ️ **NOTE** Unlike Dynamic Views, Partition Filtering will *NOT* use compute on the provider side when queried by the Delta Sharing recipient, however the provider will need to maintain **multiple shares** instead of a single share.  In addition, **column redaction or obfuscation is not supported** with this approach.  

In addition the partition values need to be known up front, an ideally not be of high cardinality.


### Inspect Partitioned Base Table
In this example note the partitioning on the base table, we will use this to filter data based upon the recipient.

In [0]:
-- Check partitioning on base table
DESCRIBE DETAIL global_retail.sales.orders_by_region;

In [0]:
-- View data by partition
SELECT * FROM global_retail.sales.orders_by_region WHERE region = 'Europe';

### Step 1. Create Shares and Add Table Partitions to the Shares
For Partition Filtering we need to create multiple shares (one per partition value).

In [0]:
-- Create partition-specific shares
-- Share only United States partition
CREATE SHARE IF NOT EXISTS us_orders_share
COMMENT 'United States orders only';

ALTER SHARE us_orders_share 
ADD TABLE global_retail.sales.orders_by_region 
PARTITION (region = 'United States');

-- Share only Europe partition  
CREATE SHARE IF NOT EXISTS eu_orders_share
COMMENT 'Europe orders only';

ALTER SHARE eu_orders_share 
ADD TABLE global_retail.sales.orders_by_region 
PARTITION (region = 'Europe');

In [0]:
-- Describe one of the shares
DESCRIBE SHARE us_orders_share;

In [0]:
-- Show the assets in a share
SHOW ALL IN SHARE us_orders_share;

**Key Concept:** Partition filtering shares specific partitions of a table with recipients. Recipients can only access data in the shared partitions, providing storage-level access control.

### Step 2. Setup Recipients and Grant Access
Create the recipients (one per region) and grant them access to the respective share

In [0]:
-- Create recipients
CREATE RECIPIENT IF NOT EXISTS us_partner
COMMENT 'United States regional partner';

CREATE RECIPIENT IF NOT EXISTS eu_partner
COMMENT 'Europe regional partner';

CREATE RECIPIENT IF NOT EXISTS apac_partner
COMMENT 'Asia Pacific regional partner';

In [0]:
-- Grant access to partition-specific shares
GRANT SELECT ON SHARE us_orders_share TO RECIPIENT us_partner;
GRANT SELECT ON SHARE eu_orders_share TO RECIPIENT eu_partner;

### Next Steps
✅ Mount one of the shares to a catalog at a recipient (one of the recipients above)  
✅ Confirm that you can only see data for your region  

## Summary

✅ **What we accomplished:**

**Example 1 - Dynamic Views:**
- Created a view using `current_recipient()` function
- Implemented row-level security that filters data based on who is accessing it
- All partners access the same share but see different data

**Example 2 - Partition Filtering:**
- Created a partitioned table by region
- Shared specific partitions with specific recipients
- Recipients can only access their designated partition

**Comparison:**
| Pattern | Granularity | Flexibility | Performance |
|---------|-------------|-------------|-------------|
| Dynamic Views | Row-level | High (logic-based) | Good for moderate data |
| Partition Filtering | Partition-level | Low (structural) | Excellent (storage-level) |

**Next Steps:** Continue to the recipient notebook to see how these access controls work from the recipient's perspective.

&copy; 2025 Databricks, Inc. All rights reserved.